In [0]:
# creating a monitoring task for raw data gathering and creating dashboard

In [0]:
%sql
select * from mycatalog.hp_prd_data.raw_data limit 2

In [0]:
%sql
select
 processed_date,
 processed_hour,
 http_status_code,
 count(*) as total_rows,
 count(Distinct zipcode) as total_zipcodes
from mycatalog.hp_prd_data.raw_data
group by processed_date, processed_hour, http_status_code
order by processed_date desc

In [0]:
%sql
select
 count(*) as total_rows,
 count(Distinct source_url) as total_sources,
 count(Distinct zipcode) as total_zipcodes
from mycatalog.hp_prd_data.raw_data

In [0]:
# there are some duplicates do deduplication while transforming

In [0]:
# Parsing raw data

In [0]:
import requests
import json

In [0]:
import json
from pyspark.sql.functions import udf
from pyspark.sql.types import MapType, StringType

def parse_json_text(text: str):
    if text is None:
        return None
    json_text = text[4:]
    try:
        data = json.loads(json_text)
    except Exception as e:
        # You can log the error or skip
        data = None
    return data


In [0]:
import json
def parser(text: str):
    json_text = text[4:]
    try:
        data = json.loads(json_text)
    except Exception as e:
        print("unable to parse the text error ", e)
        print("response length ", len(text))
        data = None
    return data

In [0]:
df = spark.sql("select raw_json from mycatalog.hp_prd_data.raw_data limit 1").collect()[0]
response_text = df['raw_json']

In [0]:
response_text

In [0]:
raw_json = parser(response_text)

In [0]:
raw_json

In [0]:
# now we need to create pyspark code
# how to load required data from the upstream
# db we may have the options to load only.

In [0]:
# target table: mycatalog.hp_prd_data.parsed_data
try:
    last_processed_timestamp = spark

In [0]:
# timestamp calculation
    # last_processed_timestamp = spark.sql("select max(ingestion_timestamp) from mycatalog.hp_prd_data.parsed_data").collect()[0]['max']



In [0]:
%sql
select 
  inge

In [0]:
raw_df = spark.sql("select zipcode, raw_json from mycatalog.hp_prd_data.raw_data")